# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint


# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:

city_data_file = "cities.csv"

vacation_df= pd.read_csv(city_data_file)

vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,Lábrea,100,BR,1601501777,94,-7.26,-64.80,73.778,0.82
1,Rikitea,72,PF,1601501777,82,-23.12,-134.97,71.816,10.82
2,São Borja,4,BR,1601501777,46,-28.66,-56.00,85.370,2.23
3,Saint-Vallier,90,FR,1601501777,93,46.64,4.37,53.600,1.00
4,Barrow,90,US,1601501498,95,71.29,-156.79,32.000,8.20
...,...,...,...,...,...,...,...,...,...
615,Gdynia,40,PL,1601501900,87,54.52,18.53,57.992,3.10
616,Makkah al Mukarramah,0,SA,1601501808,81,21.43,39.83,79.898,0.43
617,Cedartown,0,US,1601501974,55,34.05,-85.25,73.994,0.89
618,Nushki,0,PK,1601501975,31,29.55,66.02,57.704,0.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)

In [16]:

locations = vacation_df[["Lat", "Long"]]
humidity = vacation_df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


best_weather_df = pd.DataFrame(vacation_df, columns = ["City", "Max Temp", "Wind Speed","Cloudiness"])

max_temp = (best_weather_df["Max Temp"]<=80) & (best_weather_df["Max Temp"]>70)
max_temp
best_weather_df.loc[max_temp]
best_weather_df = best_weather_df.loc[max_temp]
best_weather_df

,City,Max Temp,Wind Speed,Cloudiness
0,Lábrea,73.778,0.82,100
1,Rikitea,71.816,10.82,72
6,Avarua,78.800,9.80,21
8,Riyadh,78.800,3.10,0
10,Atuona,77.954,8.47,13
...,...,...,...,...
610,Lakota,71.960,1.71,100
612,Laurel,71.600,3.10,1
616,Makkah al Mukarramah,79.898,0.43,0
617,Cedartown,73.994,0.89,0


In [37]:
windspeed = best_weather_df["Wind Speed"] < 10
print(windspeed)

best_weather_df = best_weather_df.loc[windspeed]
best_weather_df


0      True
6      True
8      True
10     True
11     True
       ... 
610    True
612    True
616    True
617    True
619    True
Name: Wind Speed, Length: 143, dtype: bool


,City,Max Temp,Wind Speed,Cloudiness
0,Lábrea,73.778,0.82,100
6,Avarua,78.800,9.80,21
8,Riyadh,78.800,3.10,0
10,Atuona,77.954,8.47,13
11,Hong Kong,78.998,0.45,100
...,...,...,...,...
610,Lakota,71.960,1.71,100
612,Laurel,71.600,3.10,1
616,Makkah al Mukarramah,79.898,0.43,0
617,Cedartown,73.994,0.89,0


In [38]:
cloudiness = best_weather_df["Cloudiness"] == 0
print("cloundiness",cloudiness)

best_weather_df = best_weather_df.loc[cloudiness]
best_weather_df


cloundiness 0      False
6      False
8       True
10     False
11     False
       ...  
610    False
612    False
616     True
617     True
619    False
Name: Cloudiness, Length: 143, dtype: bool


,City,Max Temp,Wind Speed,Cloudiness
8,Riyadh,78.800,3.10,0
13,Nouadhibou,71.600,7.20,0
54,Buenos Aires,75.002,4.10,0
91,Morondava,75.902,3.99,0
99,Kailāras,77.558,2.64,0
129,Domoni,76.028,0.43,0
149,Caravelas,74.282,2.85,0
210,Itarema,78.224,7.47,0
260,Ibrā’,73.058,0.81,0
285,Mahajanga,78.800,2.60,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
